# CARLA Client - Clean Environment Setup with Autopilot Car (Synchronous Mode)

This notebook demonstrates:
1. Connecting to CARLA simulator in **synchronous mode**
2. Removing all unnecessary objects from the map (keeping only road, car, pavement)
3. Spawning a vehicle with a camera
4. Enabling autopilot mode
5. Setting up third-person spectator view
6. Deterministic sensor data capture using queues
7. Cleanup at the end (restoring async mode)

In [25]:
import carla
import random
import time
import numpy as np
import cv2
import os
import queue
from matplotlib import cm

# Create VIRIDIS colormap for LiDAR intensity visualization
VIRIDIS = np.array(cm.get_cmap('viridis').colors)
VID_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])

print("CARLA library imported successfully!")
print("Viridis colormap loaded for LiDAR projection visualization")

CARLA library imported successfully!
Viridis colormap loaded for LiDAR projection visualization


/tmp/ipykernel_32379/3351144801.py:11: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  VIRIDIS = np.array(cm.get_cmap('viridis').colors)


## Step 1: Connect to CARLA Server

In [4]:
# Connect to the CARLA server
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

In [5]:

# Get the world
world = client.get_world()

# Get the map
carla_map = world.get_map()

# Store original settings to restore later
original_settings = world.get_settings()

# Enable synchronous mode
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.05  # 20 FPS (50ms per tick)
world.apply_settings(settings)

# Set Traffic Manager to synchronous mode as well
traffic_manager = client.get_trafficmanager(8000)
traffic_manager.set_synchronous_mode(True)

print(f"Connected to CARLA server!")
print(f"Map: {carla_map.name}")
print(f"Synchronous mode enabled with fixed_delta_seconds = {settings.fixed_delta_seconds}")

Connected to CARLA server!
Map: Carla/Maps/Town10HD_Opt
Synchronous mode enabled with fixed_delta_seconds = 0.05


## Step 2: Remove All Static Environment Objects

We'll remove all static environment objects that load with the map. This includes:
- Buildings
- Props (trees, traffic signs, street lights, etc.)
- Static meshes
- Keeping only the road surface

In [6]:
world.unload_map_layer(carla.MapLayer.Buildings)
world.unload_map_layer(carla.MapLayer.Decals)
world.unload_map_layer(carla.MapLayer.Foliage)
world.unload_map_layer(carla.MapLayer.ParkedVehicles)
world.unload_map_layer(carla.MapLayer.Particles)
world.unload_map_layer(carla.MapLayer.Props)
world.unload_map_layer(carla.MapLayer.Walls)
world.tick()

2312

## Step 3: Spawn a Vehicle with Multiple Cameras and LiDAR

In [7]:
# Get the blueprint library
blueprint_library = world.get_blueprint_library()

# Choose a vehicle blueprint (Tesla Model 3)
vehicle_bp = blueprint_library.filter('vehicle.tesla.model3')[0]

# Get a spawn point
spawn_points = carla_map.get_spawn_points()
spawn_point = random.choice(spawn_points)

# Spawn the vehicle
vehicle = world.spawn_actor(vehicle_bp, spawn_point)

print(f"Spawned vehicle: {vehicle.type_id}")
print(f"Vehicle location: {vehicle.get_location()}")
world.tick()

Spawned vehicle: vehicle.tesla.model3
Vehicle location: Location(x=0.000000, y=0.000000, z=0.000000)


2313

In [8]:
# Get camera blueprint
camera_bp = blueprint_library.find('sensor.camera.rgb')

# Set camera attributes
camera_bp.set_attribute('image_size_x', '1920')
camera_bp.set_attribute('image_size_y', '1080')
camera_bp.set_attribute('fov', '90')

# Define camera configurations: (name, location, rotation)
camera_configs = {
    'front': {
        'transform': carla.Transform(
            carla.Location(x=2.5, z=1.0),  # Front of vehicle
            carla.Rotation(pitch=0, yaw=0)  # Looking forward
        )
    },
    'left': {
        'transform': carla.Transform(
            carla.Location(x=0.0, y=-1.0, z=1.0),  # Left side
            carla.Rotation(pitch=0, yaw=-90)  # Looking left
        )
    },
    'right': {
        'transform': carla.Transform(
            carla.Location(x=0.0, y=1.0, z=1.0),  # Right side
            carla.Rotation(pitch=0, yaw=90)  # Looking right
        )
    }
}

# Spawn all cameras
cameras = {}
for cam_name, config in camera_configs.items():
    cam = world.spawn_actor(camera_bp, config['transform'], attach_to=vehicle)
    cameras[cam_name] = cam
    print(f"Camera '{cam_name}' attached to vehicle at {config['transform'].location}")

print(f"\nTotal cameras attached: {len(cameras)}")
world.tick()

Camera 'front' attached to vehicle at Location(x=2.500000, y=0.000000, z=1.000000)
Camera 'left' attached to vehicle at Location(x=0.000000, y=-1.000000, z=1.000000)
Camera 'right' attached to vehicle at Location(x=0.000000, y=1.000000, z=1.000000)

Total cameras attached: 3


2314

In [9]:
# Build camera intrinsic matrices (K) for LiDAR projection
# K = [[Fx,  0, image_w/2],
#      [ 0, Fy, image_h/2],
#      [ 0,  0,         1]]

image_w = int(camera_bp.get_attribute("image_size_x").as_int())
image_h = int(camera_bp.get_attribute("image_size_y").as_int())
fov = float(camera_bp.get_attribute("fov").as_float())

# Calculate focal length from FOV
# fov = 2 * atan(image_w / (2 * focal))
# focal = image_w / (2 * tan(fov/2))
focal = image_w / (2.0 * np.tan(fov * np.pi / 360.0))

# Since pixel aspect ratio is 1, Fx = Fy
K = np.identity(3)
K[0, 0] = K[1, 1] = focal
K[0, 2] = image_w / 2.0
K[1, 2] = image_h / 2.0

print(f"Camera intrinsic matrix K computed:")
print(f"  Image size: {image_w} x {image_h}")
print(f"  FOV: {fov}°")
print(f"  Focal length: {focal:.2f} pixels")
print(f"K =\n{K}")

Camera intrinsic matrix K computed:
  Image size: 1920 x 1080
  FOV: 90.0°
  Focal length: 960.00 pixels
K =
[[960.   0. 960.]
 [  0. 960. 540.]
 [  0.   0.   1.]]


In [15]:
# Get LiDAR blueprint
lidar_bp = blueprint_library.find('sensor.lidar.ray_cast')

# Set LiDAR attributes
lidar_bp.set_attribute('channels', '64')
lidar_bp.set_attribute('points_per_second', '1200000')
lidar_bp.set_attribute('rotation_frequency', '20')
lidar_bp.set_attribute('range', '100')
lidar_bp.set_attribute('upper_fov', '10')
lidar_bp.set_attribute('lower_fov', '-30')

# Create transform for LiDAR (mounted on top of the vehicle)
lidar_transform = carla.Transform(
    carla.Location(x=0.0, z=2.5),  # On top of the vehicle
    carla.Rotation(pitch=0, yaw=0)
)

# Spawn the LiDAR attached to the vehicle
lidar = world.spawn_actor(lidar_bp, lidar_transform, attach_to=vehicle)

print(f"LiDAR attached to vehicle at {lidar_transform.location}")
world.tick()

LiDAR attached to vehicle at Location(x=0.000000, y=0.000000, z=2.500000)


2318

## Step 4: Enable Autopilot Mode

In [16]:
# Enable autopilot for the vehicle
vehicle.set_autopilot(True)

print("Autopilot enabled!")
print("Vehicle will now drive automatically")
world.tick()

Autopilot enabled!
Vehicle will now drive automatically


2319

## Step 5: Set Up Third-Person Spectator View

In [17]:
# Get the spectator
spectator = world.get_spectator()

# Function to update spectator position to follow the vehicle
def update_spectator_view():
    # Get vehicle transform
    vehicle_transform = vehicle.get_transform()
    
    # Calculate spectator position (behind and above the vehicle)
    spectator_transform = carla.Transform(
        vehicle_transform.location + carla.Location(x=0, z=4),
        carla.Rotation(pitch=-15, yaw=vehicle_transform.rotation.yaw)
    )
    
    # Set spectator transform
    spectator.set_transform(spectator_transform)

# Initial spectator position
update_spectator_view()

print("Third-person spectator view set!")
print("The spectator camera is following the vehicle")
world.tick()

Third-person spectator view set!
The spectator camera is following the vehicle


2320

## Step 6: Set Up Sensor Recording with LiDAR-to-Camera Projection (Synchronous Mode)

This sets up recording from all 3 cameras (front, left, right) and the LiDAR sensor with **LiDAR-to-Camera Projection**:
- All sensors from the same simulation tick share the **same frame number**
- **LiDAR points are projected onto each camera image** using camera intrinsics and transforms
- Projected points are colored by intensity using the Viridis colormap
- Fusion metadata JSON files link camera images + LiDAR for each frame
- Ensures temporal synchronization for downstream fusion algorithms

In [22]:

# Create output directories
output_base_dir = "carla_output"
camera_dirs = {}
for cam_name in cameras.keys():
    cam_dir = os.path.join(output_base_dir, f"camera_{cam_name}")
    os.makedirs(cam_dir, exist_ok=True)
    camera_dirs[cam_name] = cam_dir
    print(f"Created output directory: {cam_dir}")

lidar_dir = os.path.join(output_base_dir, "lidar")
os.makedirs(lidar_dir, exist_ok=True)
print(f"Created output directory: {lidar_dir}")

fusion_dir = os.path.join(output_base_dir, "fusion")
os.makedirs(fusion_dir, exist_ok=True)
print(f"Created output directory: {fusion_dir}")

projection_dir = os.path.join(output_base_dir, "lidar_projection")
os.makedirs(projection_dir, exist_ok=True)
print(f"Created output directory: {projection_dir}")

# Queues for synchronous sensor data collection
camera_queues = {cam_name: queue.Queue() for cam_name in cameras.keys()}
lidar_queue = queue.Queue()

# Unified frame counter
frame_counter = 0

# Capture interval
fixed_delta = world.get_settings().fixed_delta_seconds
capture_interval_seconds = 0.5
capture_interval_ticks = int(capture_interval_seconds / fixed_delta)

# Projection settings
DOT_EXTENT = 2

def create_image_callback(cam_name):
    """Create a callback function for each camera"""
    def process_image(image):
        camera_queues[cam_name].put(image)
    return process_image

def lidar_callback(point_cloud):
    """Callback that puts LiDAR data in a queue"""
    lidar_queue.put(point_cloud)

def project_lidar_to_camera(lidar_data, camera, im_array):
    """Project LiDAR points onto camera image"""
    # Get point cloud data
    p_cloud_size = len(lidar_data)
    p_cloud = np.copy(np.frombuffer(lidar_data.raw_data, dtype=np.dtype('f4')))
    p_cloud = np.reshape(p_cloud, (p_cloud_size, 4))
    
    # Extract intensity
    intensity = np.array(p_cloud[:, 3])
    
    # Point cloud in lidar sensor space
    local_lidar_points = np.array(p_cloud[:, :3]).T
    local_lidar_points = np.r_[local_lidar_points, [np.ones(local_lidar_points.shape[1])]]
    
    # Transform to world space
    lidar_2_world = lidar.get_transform().get_matrix()
    world_points = np.dot(lidar_2_world, local_lidar_points)
    
    # Transform to camera space
    world_2_camera = np.array(camera.get_transform().get_inverse_matrix())
    sensor_points = np.dot(world_2_camera, world_points)
    
    # Convert coordinate system
    point_in_camera_coords = np.array([
        sensor_points[1],
        sensor_points[2] * -1,
        sensor_points[0]])
    
    # Project to 2D
    points_2d = np.dot(K, point_in_camera_coords)
    points_2d = np.array([
        points_2d[0, :] / points_2d[2, :],
        points_2d[1, :] / points_2d[2, :],
        points_2d[2, :]])
    
    # Filter points
    points_2d = points_2d.T
    intensity = intensity.T
    
    points_in_canvas_mask = \
        (points_2d[:, 0] > 0.0) & (points_2d[:, 0] < image_w) & \
        (points_2d[:, 1] > 0.0) & (points_2d[:, 1] < image_h) & \
        (points_2d[:, 2] > 0.0)
    
    points_2d = points_2d[points_in_canvas_mask]
    intensity = intensity[points_in_canvas_mask]
    
    # Get coordinates
    u_coord = points_2d[:, 0].astype(int)
    v_coord = points_2d[:, 1].astype(int)
    
    # Normalize intensity
    intensity = 4 * intensity - 3
    intensity = np.clip(intensity, 0.0, 1.0)
    intensity = 1.0 - intensity
    
    # Map to colors
    color_map = np.array([
        np.interp(intensity, VID_RANGE, VIRIDIS[:, 2]) * 255.0,
        np.interp(intensity, VID_RANGE, VIRIDIS[:, 1]) * 255.0,
        np.interp(intensity, VID_RANGE, VIRIDIS[:, 0]) * 255.0
    ]).astype(np.uint8).T
    
    # Draw points
    if DOT_EXTENT <= 0:
        im_array[v_coord, u_coord] = color_map
    else:
        for i in range(len(points_2d)):
            v_min = max(0, v_coord[i] - DOT_EXTENT)
            v_max = min(image_h, v_coord[i] + DOT_EXTENT + 1)
            u_min = max(0, u_coord[i] - DOT_EXTENT)
            u_max = min(image_w, u_coord[i] + DOT_EXTENT + 1)
            im_array[v_min:v_max, u_min:u_max] = color_map[i]
    
    return im_array, len(points_2d)

def save_camera_image(cam_name, image, frame_num):
    """Save camera image to disk"""
    array = np.copy(np.frombuffer(image.raw_data, dtype=np.dtype("uint8")))
    array = np.reshape(array, (image.height, image.width, 4))
    array = array[:, :, :3]
    
    filename = os.path.join(camera_dirs[cam_name], f"frame_{frame_num:06d}.jpg")
    cv2.imwrite(filename, array)
    
    return filename, array

def save_projected_image(cam_name, im_array, frame_num):
    """Save camera image with LiDAR projection overlay"""
    proj_cam_dir = os.path.join(projection_dir, f"camera_{cam_name}")
    os.makedirs(proj_cam_dir, exist_ok=True)
    
    filename = os.path.join(proj_cam_dir, f"frame_{frame_num:06d}.jpg")
    cv2.imwrite(filename, im_array)
    
    return filename

def save_lidar_data(point_cloud, frame_num):
    """Save LiDAR point cloud to disk"""
    points = np.frombuffer(point_cloud.raw_data, dtype=np.dtype('f4'))
    points = np.reshape(points, (-1, 4))
    
    filename = os.path.join(lidar_dir, f"lidar_{frame_num:06d}.npy")
    np.save(filename, points)
    
    return filename, len(points)

def save_fusion_metadata(frame_num, camera_files, projection_files, lidar_file, lidar_points, projection_counts, timestamp):
    """Save metadata associating camera images with LiDAR scan"""
    import json
    
    metadata = {
        "frame_id": frame_num,
        "timestamp": timestamp,
        "cameras": camera_files,
        "lidar_projections": projection_files,
        "projection_point_counts": projection_counts,
        "lidar": {
            "file": lidar_file,
            "num_points": lidar_points
        }
    }
    
    metadata_file = os.path.join(fusion_dir, f"frame_{frame_num:06d}.json")
    with open(metadata_file, 'w') as f:
        json.dump(metadata, f, indent=2)
    
    return metadata

def process_sensor_queues_with_projections(should_save=False, sim_time=0.0):
    """Process sensor queues and return projection images for AI controller"""
    global frame_counter
    
    if not should_save:
        for cam_name in cameras.keys():
            while not camera_queues[cam_name].empty():
                camera_queues[cam_name].get()
        while not lidar_queue.empty():
            lidar_queue.get()
        return None, {}
    
    # Collect sensor data
    camera_images = {}
    lidar_data = None
    
    for cam_name in cameras.keys():
        if not camera_queues[cam_name].empty():
            camera_images[cam_name] = camera_queues[cam_name].get()
            while not camera_queues[cam_name].empty():
                camera_queues[cam_name].get()
    
    if not lidar_queue.empty():
        lidar_data = lidar_queue.get()
        while not lidar_queue.empty():
            lidar_queue.get()
    
    projection_images = {}
    
    if len(camera_images) == len(cameras) and lidar_data is not None:
        current_frame = frame_counter
        frame_counter += 1
        
        # Save camera images
        camera_files = {}
        camera_arrays = {}
        for cam_name, image in camera_images.items():
            filename, array = save_camera_image(cam_name, image, current_frame)
            camera_files[cam_name] = filename
            camera_arrays[cam_name] = array
        
        # Save LiDAR data
        lidar_file, num_points = save_lidar_data(lidar_data, current_frame)
        
        # Project LiDAR onto cameras
        projection_files = {}
        projection_counts = {}
        for cam_name in cameras.keys():
            proj_array = camera_arrays[cam_name].copy()
            proj_array, num_projected = project_lidar_to_camera(
                lidar_data, cameras[cam_name], proj_array)
            
            proj_filename = save_projected_image(cam_name, proj_array, current_frame)
            projection_files[cam_name] = proj_filename
            projection_counts[cam_name] = num_projected
            projection_images[cam_name] = proj_array
        
        # Save metadata
        metadata = save_fusion_metadata(
            current_frame, camera_files, projection_files,
            lidar_file, num_points, projection_counts, sim_time
        )
        
        if current_frame % 10 == 0:
            total_projected = sum(projection_counts.values())
            print(f"Frame {current_frame}: Saved 3 cameras + LiDAR ({num_points} points) + Projections ({total_projected} projected)")
        
        return metadata, projection_images
    
    return None, {}

# Attach callbacks
for cam_name, cam in cameras.items():
    cam.listen(create_image_callback(cam_name))
    print(f"Camera '{cam_name}' listener attached")

lidar.listen(lidar_callback)
print(f"LiDAR listener attached")

print(f"\nAll sensors ready for synchronous recording with LIDAR-TO-CAMERA PROJECTION")
print(f"Capture interval: every {capture_interval_ticks} ticks ({capture_interval_seconds}s)")

Created output directory: carla_output/camera_front
Created output directory: carla_output/camera_left
Created output directory: carla_output/camera_right
Created output directory: carla_output/lidar
Created output directory: carla_output/fusion
Created output directory: carla_output/lidar_projection
Camera 'front' listener attached
Camera 'left' listener attached
Camera 'right' listener attached
LiDAR listener attached

All sensors ready for synchronous recording with LIDAR-TO-CAMERA PROJECTION
Capture interval: every 10 ticks (0.5s)


## AI Trajectory Controller

In [29]:

# AI controller state
ai_steering_history = []
ai_max_history = 5
ai_default_throttle = 0.5
ai_max_steering = 1.0

def analyze_lidar_projection(projection_image):
    """Analyze LiDAR projection to detect obstacles"""
    gray = cv2.cvtColor(projection_image, cv2.COLOR_BGR2GRAY)
    lidar_mask = gray > 10
    
    h, w = lidar_mask.shape
    zone_width = w // 3
    
    # Count points in zones
    left_zone = lidar_mask[:, :zone_width]
    center_zone = lidar_mask[:, zone_width:2*zone_width]
    right_zone = lidar_mask[:, 2*zone_width:]
    
    left_density = np.sum(left_zone) / (h * zone_width)
    center_density = np.sum(center_zone) / (h * zone_width)
    right_density = np.sum(right_zone) / (h * zone_width)
    
    # Focus on bottom half (closer objects)
    bottom_half = lidar_mask[h//2:, :]
    bottom_left = bottom_half[:, :zone_width]
    bottom_center = bottom_half[:, zone_width:2*zone_width]
    bottom_right = bottom_half[:, 2*zone_width:]
    
    close_left_density = np.sum(bottom_left) / (h//2 * zone_width)
    close_center_density = np.sum(bottom_center) / (h//2 * zone_width)
    close_right_density = np.sum(bottom_right) / (h//2 * zone_width)
    
    return {
        'left_density': left_density,
        'center_density': center_density,
        'right_density': right_density,
        'close_left_density': close_left_density,
        'close_center_density': close_center_density,
        'close_right_density': close_right_density,
        'total_points': np.sum(lidar_mask)
    }

def compute_steering(analysis):
    """Compute steering angle - go straight, steer only when obstacle ahead"""
    global ai_steering_history
    
    close_center = analysis['close_center_density']
    close_left = analysis['close_left_density']
    close_right = analysis['close_right_density']
    
    # Threshold for detecting obstacle in front (increased to reduce false positives)
    obstacle_threshold = 0.15
    
    steering = 0.0
    
    # Only steer if there's an obstacle directly ahead
    if close_center > obstacle_threshold:
        # Object in front! Steer toward the clearer side
        # Add a small margin to prefer going straight when sides are similar
        if close_left < close_right - 0.02:
            # Left is clearly clearer, steer left
            steering = -0.5
        elif close_right < close_left - 0.02:
            # Right is clearly clearer, steer right
            steering = 0.5
        else:
            # Both sides similar, prefer slight left to avoid right bias
            steering = -0.3
    else:
        # No obstacle ahead - go straight
        steering = 0.0
    
    # Smooth steering with history
    ai_steering_history.append(steering)
    if len(ai_steering_history) > ai_max_history:
        ai_steering_history.pop(0)
    
    smoothed_steering = np.mean(ai_steering_history)
    
    return np.clip(smoothed_steering, -ai_max_steering, ai_max_steering)

def compute_throttle_brake(analysis):
    """Always move forward, slow down only for very close obstacles"""
    close_center = analysis['close_center_density']
    
    # Very high threshold - only slow down for very close obstacles
    danger_threshold = 0.25
    
    if close_center > danger_threshold:
        # Very close obstacle - slow down but keep moving
        throttle = 0.3
        brake = 0.0
    else:
        # Normal forward motion
        throttle = ai_default_throttle
        brake = 0.0
    
    return throttle, brake

def get_ai_control(lidar_projection_images):
    """Main AI control function - returns vehicle control based on LiDAR projections"""
    front_projection = lidar_projection_images.get('front')
    
    if front_projection is None:
        control = carla.VehicleControl()
        control.throttle = 0.0
        control.brake = 1.0
        control.steer = 0.0
        return control, {}
    
    # Analyze projection
    analysis = analyze_lidar_projection(front_projection)
    
    # Compute controls
    steering = compute_steering(analysis)
    throttle, brake = compute_throttle_brake(analysis)
    
    # Create control
    control = carla.VehicleControl()
    control.throttle = throttle
    control.brake = brake
    control.steer = steering
    control.hand_brake = False
    control.manual_gear_shift = False
    
    return control, analysis

print("\n" + "="*60)
print("AI TRAJECTORY CONTROLLER READY")
print("="*60)
print("Vehicle will be controlled by AI based on LiDAR projections")



AI TRAJECTORY CONTROLLER READY
Vehicle will be controlled by AI based on LiDAR projections


## Step 7: Run Simulation with Continuous Spectator Updates (Synchronous)

The vehicle will drive on autopilot while the spectator follows it in third-person view.
In synchronous mode, the simulation advances only when `world.tick()` is called.

In [30]:

# DISABLE AUTOPILOT - AI takes control
vehicle.set_autopilot(False)
print("\nAutopilot DISABLED - AI controller taking over")

simulation_duration = 30
total_ticks = int(simulation_duration / fixed_delta)

print(f"\nRunning AI-controlled simulation for {simulation_duration} seconds ({total_ticks} ticks)...")
print("Watch the CARLA simulator window to see AI driving!\n")

tick_count = 0
last_print_time = 0
current_projection_images = {}

try:
    while tick_count < total_ticks:
        # Advance simulation
        world.tick()
        tick_count += 1
        
        # Wait for sensor data
        timeout = 2.0
        start_time = time.time()
        
        while time.time() - start_time < timeout:
            all_ready = True
            for cam_name in cameras.keys():
                if camera_queues[cam_name].qsize() == 0:
                    all_ready = False
                    break
            if lidar_queue.qsize() == 0:
                all_ready = False
            
            if all_ready:
                break
            time.sleep(0.001)
        
        # Process sensor data
        should_save = (tick_count % capture_interval_ticks == 0)
        sim_time = tick_count * fixed_delta
        metadata, projection_images = process_sensor_queues_with_projections(should_save, sim_time)
        
        # Update projection images for AI
        if projection_images:
            current_projection_images = projection_images
        
        # AI CONTROL
        if current_projection_images:
            control, analysis = get_ai_control(current_projection_images)
            vehicle.apply_control(control)
            
            # Print AI decisions
            if tick_count % 20 == 0:
                print(f"  AI Control - Steer: {control.steer:+.3f}, "
                      f"Throttle: {control.throttle:.2f}, "
                      f"Brake: {control.brake:.2f}, "
                      f"Center density: {analysis['close_center_density']:.3f}")
        
        # Update spectator
        update_spectator_view()
        
        # Get vehicle info
        vehicle_location = vehicle.get_location()
        vehicle_velocity = vehicle.get_velocity()
        speed_kmh = 3.6 * np.sqrt(vehicle_velocity.x**2 + vehicle_velocity.y**2 + vehicle_velocity.z**2)
        
        # Print status
        current_sim_time = tick_count * fixed_delta
        if int(current_sim_time) % 5 == 0 and int(current_sim_time) != last_print_time:
            last_print_time = int(current_sim_time)
            print(f"\nTime: {int(current_sim_time)}s | "
                  f"Speed: {speed_kmh:.1f} km/h | "
                  f"Location: ({vehicle_location.x:.1f}, {vehicle_location.y:.1f}, {vehicle_location.z:.1f})")
        
except KeyboardInterrupt:
    print("\nSimulation interrupted by user")

print("\nAI-controlled simulation completed!")
print(f"Vehicle drove autonomously using LiDAR projection analysis")


Autopilot DISABLED - AI controller taking over

Running AI-controlled simulation for 30 seconds (600 ticks)...
Watch the CARLA simulator window to see AI driving!

  AI Control - Steer: -0.300, Throttle: 0.30, Brake: 0.00, Center density: 1.000
  AI Control - Steer: -0.300, Throttle: 0.30, Brake: 0.00, Center density: 1.000
  AI Control - Steer: -0.300, Throttle: 0.30, Brake: 0.00, Center density: 1.000
  AI Control - Steer: -0.300, Throttle: 0.30, Brake: 0.00, Center density: 1.000
Frame 130: Saved 3 cameras + LiDAR (24024 points) + Projections (15052 projected)
  AI Control - Steer: -0.300, Throttle: 0.30, Brake: 0.00, Center density: 1.000

Time: 5s | Speed: 18.4 km/h | Location: (-41.5, -10.5, 0.0)
  AI Control - Steer: -0.300, Throttle: 0.30, Brake: 0.00, Center density: 1.000
  AI Control - Steer: -0.300, Throttle: 0.30, Brake: 0.00, Center density: 1.000
  AI Control - Steer: -0.300, Throttle: 0.30, Brake: 0.00, Center density: 1.000
  AI Control - Steer: -0.300, Throttle: 0.30

## Step 8: Cleanup - Destroy All Spawned Actors

This will remove the vehicle, cameras, and LiDAR we created, and restore asynchronous mode.

In [31]:
# Restore original settings (disable synchronous mode) before cleanup
world.apply_settings(original_settings)
traffic_manager.set_synchronous_mode(False)
print("Restored original world settings (asynchronous mode)")

# Stop all sensor listeners first
for cam_name, cam in cameras.items():
    if cam is not None:
        cam.stop()
        print(f"Stopped camera '{cam_name}' listener")

if lidar is not None:
    lidar.stop()
    print("Stopped LiDAR listener")

# Give a moment for listeners to finish
time.sleep(0.5)

# Destroy all cameras
for cam_name, cam in cameras.items():
    if cam is not None:
        cam.destroy()
        print(f"Camera '{cam_name}' destroyed")

# Destroy the LiDAR
if lidar is not None:
    lidar.destroy()
    print("LiDAR destroyed")

# Destroy the vehicle
if vehicle is not None:
    vehicle.destroy()
    print("Vehicle destroyed")

print("\nCleanup completed!")
print("All spawned actors have been removed from the simulation")
print(f"\nRecorded data saved in: {output_base_dir}/")
print(f"  - Camera front images: {camera_dirs['front']}/")
print(f"  - Camera left images: {camera_dirs['left']}/")
print(f"  - Camera right images: {camera_dirs['right']}/")
print(f"  - LiDAR point clouds: {lidar_dir}/")
print(f"  - LiDAR projections: {projection_dir}/")
print(f"  - Fusion metadata: {fusion_dir}/")
world.tick()

Restored original world settings (asynchronous mode)
Stopped camera 'front' listener
Stopped camera 'left' listener
Stopped camera 'right' listener
Stopped LiDAR listener
Camera 'front' destroyed
Camera 'left' destroyed
Camera 'right' destroyed
LiDAR destroyed
Vehicle destroyed

Cleanup completed!
All spawned actors have been removed from the simulation

Recorded data saved in: carla_output/
  - Camera front images: carla_output/camera_front/
  - Camera left images: carla_output/camera_left/
  - Camera right images: carla_output/camera_right/
  - LiDAR point clouds: carla_output/lidar/
  - LiDAR projections: carla_output/lidar_projection/
  - Fusion metadata: carla_output/fusion/


4084

## Optional Tweaks

Disable Autopilot

In [ ]:
vehicle.set_autopilot(False)

print("Autopilot disabled!")
world.tick()